In [1]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import cs_vqe as c
import itertools
import qubit_conversion as q_conv

In [17]:
import ast
import matplotlib
import matplotlib.pyplot as plt

f = open("hamiltonians.txt","r")
hamiltonians = ast.literal_eval(f.read())
f.close()

for h in hamiltonians.keys():
    print(h, '# qubits:', hamiltonians[h][1])

H2-S1_STO-3G_singlet # qubits: 18
C1-O1_STO-3G_singlet # qubits: 16
H1-Cl1_STO-3G_singlet # qubits: 16
H1-Na1_STO-3G_singlet # qubits: 16
H2-Mg1_STO-3G_singlet # qubits: 17
H1-F1_3-21G_singlet # qubits: 18
H1-Li1_3-21G_singlet # qubits: 18
Be1_STO-3G_singlet # qubits: 5
H1-F1_STO-3G_singlet # qubits: 8
H1-Li1_STO-3G_singlet # qubits: 8
Ar1_STO-3G_singlet # qubits: 13
F2_STO-3G_singlet # qubits: 15
H1-O1_STO-3G_singlet # qubits: 8
H2-Be1_STO-3G_singlet # qubits: 9
H2-O1_STO-3G_singlet # qubits: 10
H2_3-21G_singlet # qubits: 5
H2_6-31G_singlet # qubits: 5
H3-N1_STO-3G_singlet # qubits: 13
H4-C1_STO-3G_singlet # qubits: 14
Mg1_STO-3G_singlet # qubits: 13
N2_STO-3G_singlet # qubits: 15
Ne1_STO-3G_singlet # qubits: 5
O2_STO-3G_singlet # qubits: 15
H1-Li1-O1_STO-3G_singlet # qubits: 18
H1-He1_STO-3G_singlet # qubits: 2
H3_STO-3G_singlet_1+ # qubits: 3
H1-He1_3-21G_singlet_1+ # qubits: 6
H3_3-21G_singlet_1+ # qubits: 9
H4-N1_STO-3G_singlet_1+ # qubits: 14


In [61]:
speciesname = 'H1-He1_3-21G_singlet_1+'

encoding = hamiltonians[speciesname][0] # in this dataset, always 'JW' for Jordan-Wigner, but leaves room for trying Bravyi-Kitaev as well
num_qubits = hamiltonians[speciesname][1] # number of qubits (all of these Hamiltonians have been tapered for molecular symmetries)
ham = hamiltonians[speciesname][2] # full Hamiltonian
ham_noncon = hamiltonians[speciesname][3] # noncontextual part of Hamiltonian, found by greedy DFS
true_gs = hamiltonians[speciesname][4] # ground state energy of full Hamiltonian (in Hartree)
gs_noncon = hamiltonians[speciesname][5] # list containing information about noncontextual ground state: zeroth entry is ground state energy of noncontextual part of Hamiltonian
#print('commuting generators:',model[0], '\n')
#print('anticommuting generators:',model[1], '\n')
#print('term reconstruction:',model[2], '\n')

print('number of qubits:', num_qubits)
print('true ground state energy:', true_gs)
print('noncontextual approximation to ground state energy:', gs_noncon[0])
print(ham_noncon)

number of qubits: 6
true ground state energy: -4.259188160411326
noncontextual approximation to ground state energy: -4.235679984712036
{'IZIZIZ': 1.137117313013182, 'ZIZIZI': 1.137117313013182, 'ZIIIII': -0.6757846691545434, 'IZIIII': -0.6757846691545433, 'IIIIZZ': 0.24918329066787195, 'IIIZII': -0.24459538421117766, 'IIZIII': -0.24459538421117766, 'ZZZZZZ': 0.2324217371047461, 'IZIZZZ': -0.23101530963627243, 'ZIZIZZ': -0.23101530963627243, 'IIIIII': -0.22683086324346313, 'ZZIIII': 0.17119154268004644, 'IZZIII': 0.14145232769859284, 'ZIIZII': 0.14145232769859284, 'IZIIZI': 0.1411974641913617, 'ZIIIIZ': 0.1411974641913617, 'IIIIIZ': 0.13839367136530806, 'IIIIZI': 0.13839367136530806, 'IIZZII': 0.13734855781563715, 'IIIZZI': 0.1365862143604441, 'IIZIIZ': 0.1365862143604441, 'IZZZIZ': -0.12223086443933409, 'ZIZZZI': -0.12223086443933409, 'ZZIZIZ': -0.11934196603424856, 'ZZZIZI': -0.11934196603424856, 'IXZXII': -0.0627599937687078, 'IYZYII': -0.0627599937687078, 'IZIZII': -0.0620181618623

In [41]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)
gs_noncon = c.find_gs_noncon(ham_noncon)
gs_noncon_energy = gs_noncon[0]
ep_state = gs_noncon[1]
ham_context = {p:c for p,c in ham.items() if p not in ham_noncon}

In [54]:
model[0:2], gs_noncon_energy, ep_state

((['ZIZIZI', 'IZIZIZ', 'IIZIZI', 'IIIIZZ', 'IIIIIZ'], ['IZIIII', 'IXZXII']),
 -4.235679984712036,
 [[-1, -1, -1, 1, -1], [0.9999999999999987, 5.068707371578793e-08]])

# The epistemic state defines a classical probability distribution:

In [44]:
def ontic_prob(ep_state, ontic_state):
    
    if ep_state[0] != ontic_state[0]:
        return 0
    
    else:
        prod = 1
        for index, r in enumerate(ep_state[1]):
            f = 1/2 * abs(r + ontic_state[1][index])
            prod *= f
        
        return prod    

def epistemic_dist(ep_state):
    size_G = len(ep_state[0])
    size_Ci = len(ep_state[1])
    size_R = size_G + size_Ci
    
    ep_prob = {}
    
    ontic_states = list(itertools.product([1, -1], repeat=size_R))
    
    for o in ontic_states:
        o_state = [list(o[0:size_G]), list(o[size_G:size_R])]
        o_prob = ontic_prob(ep_state, o_state)
        
        if o_prob != 0:
            ep_prob[o] = o_prob
    
    return ep_prob

In [45]:
epistemic_dist(ep_state)

{(-1, -1, -1, 1, -1, 1, 1): 0.5000000253435365,
 (-1, -1, -1, 1, -1, 1, -1): 0.4999999746564628,
 (-1, -1, -1, 1, -1, -1, 1): 3.3306692426973383e-16,
 (-1, -1, -1, 1, -1, -1, -1): 3.3306689050536005e-16}

# Find a simultaneous eigenstate of the operators G_i and A:

In [127]:
r1 = ep_state[1][0]
r2 = ep_state[1][1]
t = np.arctan((r1 + 1) / r2)
t

1.5707963014513597

In [128]:
def bin_to_int(bin_str):
    bits = [int(b) for b in bin_str]
    for index, b in enumerate(bits):
        bits[index] = b * 2 ** (len(bits)-index-1)
    return sum(bits)

psi = [0 for i in range(2**num_qubits)]
psi[bin_to_int('000011')] = np.sin(t)
psi[bin_to_int('010111')] = np.cos(t)
print(psi)

[0, 0, 0, 0.9999999999999997, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.534353688593914e-08, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [139]:
from openfermion.linalg import LinearQubitOperator

psi = np.array(psi)
ham_q = q_conv.dict_to_QubitOperator(ham, num_qubits)
ham_noncon_q = q_conv.dict_to_QubitOperator(ham_noncon, num_qubits)
ham_context_q = q_conv.dict_to_QubitOperator(ham_context, num_qubits)

#take expectation value algebraically
H = LinearQubitOperator(ham_q, num_qubits)
H_psi = H.matvec(psi)
expect = psi.dot(H_psi)

print('<H> w.r.t. simultaneous eigenstate:', expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match? 

<H> w.r.t. simultaneous eigenstate: (-4.235679984712033+0j)
Noncontextual approximation: -4.235679984712036
Expectation values match? True


# Do we have any quantum corrections?

In [172]:
G = model[0]
terms_context = list(ham_context.keys())
G

['ZIZIZI', 'IZIZIZ', 'IIZIZI', 'IIIIZZ', 'IIIIIZ']

In [179]:
q_corr_terms = []
for t in terms_context:
    commutes = []
    for g in G:
        if c.commute(t, g):
            commutes.append(g)
    print(t, 'commutes with the noncontextual generators:', commutes)
    if commutes == G:
        q_corr_terms.append(t)
        
q_corr_terms

IIIIIX commutes with the noncontextual generators: ['ZIZIZI', 'IIZIZI']
IIIIXI commutes with the noncontextual generators: ['IZIZIZ', 'IIIIIZ']
IIIIXZ commutes with the noncontextual generators: ['IZIZIZ', 'IIIIIZ']
IIIIYY commutes with the noncontextual generators: ['IIIIZZ']
IIIXIX commutes with the noncontextual generators: ['ZIZIZI', 'IZIZIZ', 'IIZIZI']
IIIXXI commutes with the noncontextual generators: ['IIIIIZ']
IIIXYY commutes with the noncontextual generators: ['IZIZIZ', 'IIIIZZ']
IIIXZX commutes with the noncontextual generators: ['ZIZIZI', 'IZIZIZ', 'IIZIZI']
IIIXZZ commutes with the noncontextual generators: ['ZIZIZI', 'IIZIZI', 'IIIIZZ', 'IIIIIZ']
IIIYIY commutes with the noncontextual generators: ['ZIZIZI', 'IZIZIZ', 'IIZIZI']
IIIYYX commutes with the noncontextual generators: ['IZIZIZ', 'IIIIZZ']
IIIYZY commutes with the noncontextual generators: ['ZIZIZI', 'IZIZIZ', 'IIZIZI']
IIIZXZ commutes with the noncontextual generators: ['IZIZIZ', 'IIIIIZ']
IIXIXI commutes with the

[]

In [101]:
psi2 = [0 for i in range(2**num_qubits)]
psi2[bin_to_int('000011')] = np.sin(t)
psi2[bin_to_int('010111')] = -np.cos(t)

In [121]:
from qiskit import QuantumCircuit
from qiskit.extensions import Initialize
from qiskit.circuit import Parameter

anz = QuantumCircuit(num_qubits) # We are redefining qc
anz.initialize(psi)
anz.ry(Parameter('x'), 3)
anz.initialize(psi2)
anz.rx(Parameter('y'), 3)

In [122]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.opflow.primitive_ops import PauliOp

ham_qiskit = sum([PauliOp(Pauli(k), ham[k]) for k in ham.keys()])

seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
slsqp = SLSQP(maxiter=1000)

vqe = VQE(anz, optimizer=slsqp, quantum_instance=qi)
vqe_run    = vqe.compute_minimum_eigenvalue(operator=ham_qiskit)
vqe_result = vqe_run.optimal_value# + shift

print('VQE:', vqe_result, '|', 'Noncontextual Ground State:', gs_noncon_energy, 'True Ground State:', true_gs)

VQE: -4.235679984707565 | Noncontextual Ground State: -4.235679984712036 True Ground State: -4.259188160411326


In [180]:
from qiskit.aqua.algorithms import NumPyEigensolver

order = [0,1,2,3,4,5,6,7]
print(c.contextual_subspace_approximations(ham,model,fn_form,ep_state,order))

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
exact_energy = np.real(result.eigenvalues)

print(exact_energy)

[-4.235679984712035, -4.235679984712034, -4.235679984712036, -4.2356799847120365, -4.235679984712036, -4.2572886431457135, -4.259188160411331, -4.259188160411323, -4.2591881604113135]
[-4.25918816]
